## load data

In [1]:
import pandas as pd
import numpy as np
import os
print(os.getcwd())
# credit_card_balance.csv installments_payments.csv

# load origin data
df_orig_cc_bal = pd.read_csv("../../data/credit_card_balance.csv")
df_orig_cc_bal.head()
df_orig_cc_ins = pd.read_csv("../../data/installments_payments.csv")
df_orig_cc_ins.head()


/Users/long.li/Documents/project/kaggle/kaggle/homecredit/long.li/eda


,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In [2]:
# prepare data for eda and load description data
print(df_orig_cc_bal.columns)
print(df_orig_cc_ins.columns)

print("df_orig_cc_bal shape:", df_orig_cc_bal.shape)
print("df_orig_cc_ins shape:", df_orig_cc_ins.shape)

df_orig_cc_ins = df_orig_cc_ins.merge(df_orig_cc_bal[["SK_ID_PREV"]].drop_duplicates(keep = 'last')
                                      , how = "inner", on = ["SK_ID_PREV"])
print("\ndf_orig_cc_ins shape after merge:", df_orig_cc_ins.shape)

print("\nLoad Description data of every columns")
df_desc = pd.read_csv("../../material/HomeCredit_columns_description.csv", engine='python')
df_desc.head()

Index(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE', 'AMT_BALANCE',
       'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_ATM_CURRENT',
       'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT',
       'AMT_DRAWINGS_POS_CURRENT', 'AMT_INST_MIN_REGULARITY',
       'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT',
       'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE', 'AMT_TOTAL_RECEIVABLE',
       'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT',
       'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_POS_CURRENT',
       'CNT_INSTALMENT_MATURE_CUM', 'NAME_CONTRACT_STATUS', 'SK_DPD',
       'SK_DPD_DEF'],
      dtype='object')
Index(['SK_ID_PREV', 'SK_ID_CURR', 'NUM_INSTALMENT_VERSION',
       'NUM_INSTALMENT_NUMBER', 'DAYS_INSTALMENT', 'DAYS_ENTRY_PAYMENT',
       'AMT_INSTALMENT', 'AMT_PAYMENT'],
      dtype='object')
df_orig_cc_bal shape: (3840312, 23)
df_orig_cc_ins shape: (13605401, 8)

df_orig_cc_ins shape after merge: (4080671, 8)

Load Description data of every columns


,Unnamed: 0,Table,Row,Description,Special
0,1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,NaN
1,2,application_{train|test}.csv,TARGET,Target variable (1 - client with payment diffi...,NaN
2,5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
3,6,application_{train|test}.csv,CODE_GENDER,Gender of the client,NaN
4,7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN


## define udf

In [14]:
import numpy as np
from scipy import stats
import pandas as pd

def eda_num_base(arr, missset = [np.nan]):
    '''
    arr: numpy array as input
    output: dict with all statistics
    '''
    
    result = {}
    n = len(arr)
    result['n'] = n
    
    arr_nonmiss = arr[~(np.isnan(arr) | np.in1d(arr,missset))]
    n_nonmiss = len(arr_nonmiss)
    
    result['nmiss'] = result['n'] - n_nonmiss
    
    result['min'] = np.nan if n_nonmiss == 0 else min(arr_nonmiss)
    result['max'] = np.nan if n_nonmiss == 0 else max(arr_nonmiss)
    result['mean'] = np.nan if n_nonmiss == 0 else np.mean(arr_nonmiss)
    result['std'] = np.nan if n_nonmiss == 0 else np.std(arr_nonmiss, ddof = 1)
    # calculate mode
    value, count = np.unique(arr_nonmiss, return_counts = True)
    idx_mode = np.argmax(count)
    result['mode'] = np.nan if n_nonmiss == 0 else value[idx_mode]
    
    quant_point = [1, 5, 25, 50, 75, 95, 99]
    quant_arr = [np.nan] * len(quant_point) if n_nonmiss == 0 else np.percentile(arr_nonmiss, quant_point, interpolation='lower')
    
    quant_dict = dict(zip(['q%02d' % i for i in quant_point], quant_arr))
    
    result = dict(result, **quant_dict)
    
    result['pct_mode'] = count[idx_mode] / n
    result['pct_q01'] = sum(arr_nonmiss <= quant_dict['q01']) / n
    result['pct_q99'] = sum(arr_nonmiss >= quant_dict['q99']) / n
    
    # floor & cap treatment
    arr_trim = np.clip(arr_nonmiss, quant_dict['q01'], quant_dict['q99'])
    result['trim_mean'] = np.nan if n_nonmiss == 0 else np.mean(arr_trim)
    result['trim_std'] = np.nan if n_nonmiss == 0 else np.std(arr_trim, ddof = 1)
    
    return result
    
def eda_num_df(df, missset = [np.nan]):
    
    num_dict = {k: np.array(df[k], dtype = np.float64) for k in df.columns}
    eda_num_dict = [dict(eda_num_base(v), varname = k) for k, v in num_dict.items()]

    return pd.DataFrame(eda_num_dict)
    
def eda_cat_df(df, target):
    return pd.concat([eda_cat_base(v, target).assign(varname = k) for k,v in df.items()])


def eda_cat_base(x, y = None):
    '''
    y is  0/1 or continous data
    '''
    if y is not None:
        return pd.DataFrame(dict(n = y, mean = y, std = y, value = x)).groupby(['value'], as_index = False).\
        agg({'n':len, 'mean':np.mean, 'std': lambda x: np.std(x, ddof = 1)})
        
    dftmp = pd.DataFrame(dict(value = x, cnt = 1)).groupby(['value'], as_index = False).sum()
    dftmp["pct"] = dftmp["cnt"]/len(x)
    return dftmp
    value, count = np.unique(x, return_counts = True)
    percent = count/np.sum(count)
    return pd.DataFrame(dict(value = value, cnt = count, pct = percent))

class ll_eda(object):
    def __init__(self, X, y = None, numvars = None, nominal = None, ordinal = None):
        self.X = X
        self.y = y
        cols = X.columns
        dtypes = X.dtypes
        if numvars is None:
            numvars = list(cols[dtypes != 'object'])
        if nominal is None:
            nominal = list(cols[dtypes == 'object'])
        self.numvars = numvars
        self.nominal = nominal
        self.ordinal = ordinal
        
    def eda_num(self, missset = [np.nan], out = False):
        if self.numvars is None:
            self.df_eda_num = None
        else:
            dict_X = {k: np.array(self.X[k], dtype = np.float64) for k in self.numvars}
            dict_eda_num = [dict(eda_num_base(v), variable = k) for k, v in dict_X.items()]
            self.df_eda_num = pd.DataFrame(dict_eda_num)
        if out:
            return self.df_eda_num
    
    def eda_cat(self, out = False):
        if len(self.nominal) == 0:
            self.df_eda_cat = None
        else:
            dict_X = {k: np.array(self.X[k]) for k in self.nominal}
            self.df_eda_cat = pd.concat([eda_cat_base(v, self.y).assign(variable = k) for k,v in dict_X.items()])
        if out:
            return self.df_eda_cat
        
    def train(self, missset = [np.nan], out = False):
        self.eda_num(missset)
        self.eda_cat()
        if out:
            return self.df_eda_num, self.df_eda_cat
        
    def save(self, wd = ".", datadir = "data", prefix = ""):
        workpath = []
        if wd is not None:
            workpath.append(wd)
        if datadir is not None:
            workpath.append(datadir)
        if self.df_eda_num is not None:
            self.df_eda_num.to_csv("/".join(workpath + [prefix + "stat_eda_num.csv"]), index = False)
        if self.df_eda_cat is not None:
            self.df_eda_cat.to_csv("/".join(workpath + [prefix + "stat_eda_cat.csv"]), index = False)



## eda and dtype check

In [17]:
print(df_orig_cc_bal.dtypes)
print(df_orig_cc_ins.dtypes)

SK_ID_PREV                      int64
SK_ID_CURR                      int64
MONTHS_BALANCE                  int64
AMT_BALANCE                   float64
AMT_CREDIT_LIMIT_ACTUAL         int64
AMT_DRAWINGS_ATM_CURRENT      float64
AMT_DRAWINGS_CURRENT          float64
AMT_DRAWINGS_OTHER_CURRENT    float64
AMT_DRAWINGS_POS_CURRENT      float64
AMT_INST_MIN_REGULARITY       float64
AMT_PAYMENT_CURRENT           float64
AMT_PAYMENT_TOTAL_CURRENT     float64
AMT_RECEIVABLE_PRINCIPAL      float64
AMT_RECIVABLE                 float64
AMT_TOTAL_RECEIVABLE          float64
CNT_DRAWINGS_ATM_CURRENT      float64
CNT_DRAWINGS_CURRENT            int64
CNT_DRAWINGS_OTHER_CURRENT    float64
CNT_DRAWINGS_POS_CURRENT      float64
CNT_INSTALMENT_MATURE_CUM     float64
NAME_CONTRACT_STATUS           object
SK_DPD                          int64
SK_DPD_DEF                      int64
dtype: object
SK_ID_PREV                  int64
SK_ID_CURR                  int64
NUM_INSTALMENT_VERSION    float64
NUM_INSTAL

In [4]:
vars_rm = ["SK_ID_PREV", "SK_ID_CURR"]
obj_eda = ll_eda(df_orig_cc_bal[df_orig_cc_bal.columns.difference(vars_rm)])
# obj_eda = ll_eda(df_orig_cc_bal[["MONTHS_BALANCE"]])
print(obj_eda.nominal)
print(obj_eda.numvars)
obj_eda.train()
obj_eda.save(datadir = None, prefix = "cc_bal_")
# obj_eda.df_eda_cat.to_csv("cc_bal_stat_eda_cat.csv", index=False)
# obj_eda.df_eda_num.to_csv("cc_bal_stat_eda_num.csv", index=False)

['NAME_CONTRACT_STATUS']
['AMT_BALANCE', 'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT', 'AMT_DRAWINGS_POS_CURRENT', 'AMT_INST_MIN_REGULARITY', 'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT', 'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE', 'AMT_TOTAL_RECEIVABLE', 'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_POS_CURRENT', 'CNT_INSTALMENT_MATURE_CUM', 'MONTHS_BALANCE', 'SK_DPD', 'SK_DPD_DEF']


In [8]:
obj_eda = ll_eda(df_orig_cc_ins[df_orig_cc_ins.columns.difference(vars_rm)])
# obj_eda = ll_eda(df_orig_cc_bal[["MONTHS_BALANCE"]])
print(obj_eda.nominal)
print(obj_eda.numvars)
obj_eda.train()
obj_eda.save(datadir = None, prefix = "cc_ins_")

[]
['AMT_INSTALMENT', 'AMT_PAYMENT', 'DAYS_ENTRY_PAYMENT', 'DAYS_INSTALMENT', 'NUM_INSTALMENT_NUMBER', 'NUM_INSTALMENT_VERSION']


## check result of relationship of tables

In [11]:
df_stat_ccpre_by_app = df_orig_cc_bal.groupby(["SK_ID_CURR"], as_index = False).agg({"SK_ID_PREV":pd.Series.nunique})
obj_eda = ll_eda(df_stat_ccpre_by_app)
# obj_eda = ll_eda(df_orig_cc_bal[["MONTHS_BALANCE"]])
print(obj_eda.nominal)
print(obj_eda.numvars)
obj_eda.train()
obj_eda.save(datadir = None, prefix = "ccpre_acstat")

[]
['SK_ID_CURR', 'SK_ID_PREV']


In [12]:
df_stat_ccpre_by_app[df_stat_ccpre_by_app.SK_ID_PREV > 1].head()

,SK_ID_CURR,SK_ID_PREV
79,100259,2
82,100267,2
186,100594,2
229,100763,2
425,101407,2


In [14]:
df_orig_cc_bal[df_orig_cc_bal.SK_ID_CURR == 100259][["SK_ID_CURR", "SK_ID_PREV", "NAME_CONTRACT_STATUS"]].drop_duplicates(keep = 'last')

,SK_ID_CURR,SK_ID_PREV,NAME_CONTRACT_STATUS
2883919,100259,2059116,Completed
3571383,100259,1435602,Active
3746004,100259,2059116,Active


In [18]:
df_orig_cc_bal[(df_orig_cc_bal.SK_ID_CURR == 100259) & (df_orig_cc_bal.NAME_CONTRACT_STATUS == "Active")].groupby(["SK_ID_PREV"]).agg({"MONTHS_BALANCE":[min, max]})

MONTHS_BALANCE    
                      min max
SK_ID_PREV                   
1435602                -8  -2
2059116               -21 -10

In [19]:
df_orig_cc_bal[df_orig_cc_bal.NAME_CONTRACT_STATUS == "Active"]\
.groupby(["SK_ID_CURR", "MONTHS_BALANCE"], as_index = False).agg({"SK_ID_PREV":pd.Series.nunique})\
.sort_values(by = ['SK_ID_PREV'], ascending = [False]).head()

,SK_ID_CURR,MONTHS_BALANCE,SK_ID_PREV
889344,186401,-71,2
3166130,403907,-7,2
3166128,403907,-9,2
2187293,311118,-71,2
2187294,311118,-70,2


In [20]:
df_orig_cc_bal[(df_orig_cc_bal.MONTHS_BALANCE == -71) & (df_orig_cc_bal.SK_ID_CURR == 186401)]

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
2499427,1440617,186401,-71,82321.065,112500,0.0,0.0,0.0,0.0,6750.0,...,82321.065,82321.065,0.0,0,0.0,0.0,35.0,Active,0,0
3363642,2243411,186401,-71,184997.610,180000,0.0,0.0,0.0,0.0,9000.0,...,184997.610,184997.610,0.0,0,0.0,0.0,25.0,Active,0,0


In [3]:
df_orig_cc_bal[(df_orig_cc_bal.SK_ID_CURR == 100259)].sort_values(by = ['MONTHS_BALANCE']).head(10)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
2483684,2059116,100259,-21,0.0,720000,NaN,0.0,NaN,NaN,0.0,...,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
569398,2059116,100259,-20,0.0,720000,NaN,0.0,NaN,NaN,0.0,...,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
234021,2059116,100259,-19,0.0,720000,NaN,0.0,NaN,NaN,0.0,...,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
731243,2059116,100259,-18,0.0,720000,NaN,0.0,NaN,NaN,0.0,...,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
3746004,2059116,100259,-17,0.0,720000,NaN,0.0,NaN,NaN,0.0,...,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
2821314,2059116,100259,-16,0.0,720000,NaN,0.0,NaN,NaN,0.0,...,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
2206355,2059116,100259,-15,0.0,0,NaN,0.0,NaN,NaN,0.0,...,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
2110774,2059116,100259,-14,0.0,0,NaN,0.0,NaN,NaN,0.0,...,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
1287996,2059116,100259,-13,0.0,0,NaN,0.0,NaN,NaN,0.0,...,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
3199951,2059116,100259,-12,0.0,0,NaN,0.0,NaN,NaN,0.0,...,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0


账户关闭后，后续就没有记录了

## check install data

In [28]:
df_orig_cc_ins.groupby(["SK_ID_PREV", "DAYS_INSTALMENT"], as_index = False)[["SK_ID_CURR"]].agg(len)\
.sort_values(by = ['SK_ID_CURR'], ascending = [False]).head()

,SK_ID_PREV,DAYS_INSTALMENT,SK_ID_CURR
1947686,1879746,-234.0,7
1947691,1879746,-173.0,7
2879003,2331661,-120.0,6
3565386,2657947,-617.0,6
1922312,1868721,-50.0,6


In [29]:
df_orig_cc_ins[(df_orig_cc_ins.SK_ID_PREV == 1879746)&(df_orig_cc_ins.DAYS_INSTALMENT == -234)]

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
2139906,1879746,260541,0.0,51,-234.0,-244.0,35991.855,6750.000
2139912,1879746,260541,0.0,51,-234.0,-253.0,35991.855,3150.000
2139919,1879746,260541,0.0,51,-234.0,-234.0,35991.855,14841.855
2139940,1879746,260541,0.0,52,-234.0,-234.0,458.145,458.145
2139945,1879746,260541,0.0,51,-234.0,-250.0,35991.855,2250.000
2139955,1879746,260541,0.0,51,-234.0,-247.0,35991.855,4500.000
2139959,1879746,260541,0.0,51,-234.0,-240.0,35991.855,4500.000


In [30]:
df_orig_cc_ins[df_orig_cc_ins.SK_ID_PREV == 1879746].sort_values(by = ['NUM_INSTALMENT_NUMBER'])

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
2139981,1879746,260541,0.0,1,-756.0,-756.0,16875.000,16875.000
2139982,1879746,260541,0.0,2,-740.0,-740.0,334.350,334.350
2139914,1879746,260541,0.0,3,-733.0,-733.0,49500.000,49500.000
2139937,1879746,260541,0.0,4,-727.0,-727.0,25200.000,25200.000
2139910,1879746,260541,0.0,5,-691.0,-692.0,2250.000,534.150
2139988,1879746,260541,0.0,5,-691.0,-710.0,2250.000,1715.850
2139957,1879746,260541,0.0,6,-692.0,-692.0,17465.850,17465.850
2139934,1879746,260541,0.0,7,-660.0,-657.0,4876.785,4876.785
2139970,1879746,260541,0.0,8,-657.0,-657.0,73.215,73.215
2139901,1879746,260541,0.0,9,-650.0,-650.0,45000.000,45000.000


存在期数大而还款日往前的情况，估计是用户自习选择的还款时间

## 2. eda for credit balance

In [1]:
import pandas as pd
import numpy as np
import os
print(os.getcwd())
# credit_card_balance.csv installments_payments.csv

# load origin data
df_orig_pos_bal = pd.read_csv("../../data/POS_CASH_balance.csv")
df_orig_pos_bal.head()
df_orig_pos_ins = pd.read_csv("../../data/installments_payments.csv")
df_orig_pos_ins.head()


/Users/long.li/Documents/project/kaggle/kaggle/homecredit/long.li/eda


,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


### 2.1 基本字段描述性统计

In [3]:
# prepare data for eda and load description data
print(df_orig_pos_bal.columns)
print(df_orig_pos_ins.columns)

print("df_orig_cc_bal shape:", df_orig_pos_bal.shape)
print("df_orig_cc_ins shape:", df_orig_pos_ins.shape)

df_orig_pos_ins = df_orig_pos_ins.merge(df_orig_pos_bal[["SK_ID_PREV"]].drop_duplicates(keep = 'last')
                                      , how = "inner", on = ["SK_ID_PREV"])
print("\ndf_orig_cc_ins shape after merge:", df_orig_pos_ins.shape)

Index(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE', 'CNT_INSTALMENT',
       'CNT_INSTALMENT_FUTURE', 'NAME_CONTRACT_STATUS', 'SK_DPD',
       'SK_DPD_DEF'],
      dtype='object')
Index(['SK_ID_PREV', 'SK_ID_CURR', 'NUM_INSTALMENT_VERSION',
       'NUM_INSTALMENT_NUMBER', 'DAYS_INSTALMENT', 'DAYS_ENTRY_PAYMENT',
       'AMT_INSTALMENT', 'AMT_PAYMENT'],
      dtype='object')
df_orig_cc_bal shape: (10001358, 8)
df_orig_cc_ins shape: (13605401, 8)

df_orig_cc_ins shape after merge: (9522662, 8)


### 2.2 dtypes

In [5]:
print(df_orig_pos_bal.dtypes)
print(df_orig_pos_ins.dtypes)

SK_ID_PREV                 int64
SK_ID_CURR                 int64
MONTHS_BALANCE             int64
CNT_INSTALMENT           float64
CNT_INSTALMENT_FUTURE    float64
NAME_CONTRACT_STATUS      object
SK_DPD                     int64
SK_DPD_DEF                 int64
dtype: object
SK_ID_PREV                  int64
SK_ID_CURR                  int64
NUM_INSTALMENT_VERSION    float64
NUM_INSTALMENT_NUMBER       int64
DAYS_INSTALMENT           float64
DAYS_ENTRY_PAYMENT        float64
AMT_INSTALMENT            float64
AMT_PAYMENT               float64
dtype: object


### 2.3 eda

In [6]:
vars_rm = ["SK_ID_PREV", "SK_ID_CURR"]
obj_eda = ll_eda(df_orig_pos_bal[df_orig_pos_bal.columns.difference(vars_rm)])
# obj_eda = ll_eda(df_orig_cc_bal[["MONTHS_BALANCE"]])
print(obj_eda.nominal)
print(obj_eda.numvars)
obj_eda.train()
obj_eda.save(datadir = None, prefix = "pos_bal_")
# obj_eda.df_eda_cat.to_csv("cc_bal_stat_eda_cat.csv", index=False)
# obj_eda.df_eda_num.to_csv("cc_bal_stat_eda_num.csv", index=False)

['NAME_CONTRACT_STATUS']
['CNT_INSTALMENT', 'CNT_INSTALMENT_FUTURE', 'MONTHS_BALANCE', 'SK_DPD', 'SK_DPD_DEF']


In [7]:
obj_eda = ll_eda(df_orig_pos_ins[df_orig_pos_ins.columns.difference(vars_rm)])
# obj_eda = ll_eda(df_orig_cc_bal[["MONTHS_BALANCE"]])
print(obj_eda.nominal)
print(obj_eda.numvars)
obj_eda.train()
obj_eda.save(datadir = None, prefix = "pos_ins_")

[]
['AMT_INSTALMENT', 'AMT_PAYMENT', 'DAYS_ENTRY_PAYMENT', 'DAYS_INSTALMENT', 'NUM_INSTALMENT_NUMBER', 'NUM_INSTALMENT_VERSION']


### 2.4 关系确认——暂无需

## 3. EDA for previous_application

In [8]:
import pandas as pd
import numpy as np
import os
print(os.getcwd())
# credit_card_balance.csv installments_payments.csv

# load origin data
df_orig_pre_app = pd.read_csv("../../data/previous_application.csv")
df_orig_pre_app.head()


/Users/long.li/Documents/project/kaggle/kaggle/homecredit/long.li/eda


,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


### 3.1 基本字段描述性信息

In [9]:
# prepare data for eda and load description data
print(df_orig_pre_app.columns)

print("df_orig_pre_app shape:", df_orig_pre_app.shape)

Index(['SK_ID_PREV', 'SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'AMT_ANNUITY',
       'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE',
       'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START',
       'FLAG_LAST_APPL_PER_CONTRACT', 'NFLAG_LAST_APPL_IN_DAY',
       'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY',
       'RATE_INTEREST_PRIVILEGED', 'NAME_CASH_LOAN_PURPOSE',
       'NAME_CONTRACT_STATUS', 'DAYS_DECISION', 'NAME_PAYMENT_TYPE',
       'CODE_REJECT_REASON', 'NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE',
       'NAME_GOODS_CATEGORY', 'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE',
       'CHANNEL_TYPE', 'SELLERPLACE_AREA', 'NAME_SELLER_INDUSTRY',
       'CNT_PAYMENT', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION',
       'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION',
       'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL'],
      dtype='object')
df_orig_pre_app shape: (1670214, 37)


### 3.2 dtypes

In [10]:
df_orig_pre_app.dtypes

SK_ID_PREV                       int64
SK_ID_CURR                       int64
NAME_CONTRACT_TYPE              object
AMT_ANNUITY                    float64
AMT_APPLICATION                float64
AMT_CREDIT                     float64
AMT_DOWN_PAYMENT               float64
AMT_GOODS_PRICE                float64
WEEKDAY_APPR_PROCESS_START      object
HOUR_APPR_PROCESS_START          int64
FLAG_LAST_APPL_PER_CONTRACT     object
NFLAG_LAST_APPL_IN_DAY           int64
RATE_DOWN_PAYMENT              float64
RATE_INTEREST_PRIMARY          float64
RATE_INTEREST_PRIVILEGED       float64
NAME_CASH_LOAN_PURPOSE          object
NAME_CONTRACT_STATUS            object
DAYS_DECISION                    int64
NAME_PAYMENT_TYPE               object
CODE_REJECT_REASON              object
NAME_TYPE_SUITE                 object
NAME_CLIENT_TYPE                object
NAME_GOODS_CATEGORY             object
NAME_PORTFOLIO                  object
NAME_PRODUCT_TYPE               object
CHANNEL_TYPE             

### 3.3 eda

In [15]:
vars_rm = ["SK_ID_PREV", "SK_ID_CURR"]
obj_eda = ll_eda(df_orig_pre_app[df_orig_pre_app.columns.difference(vars_rm)])
# obj_eda = ll_eda(df_orig_cc_bal[["MONTHS_BALANCE"]])
print(obj_eda.nominal)
print(obj_eda.numvars)
obj_eda.train()
obj_eda.save(datadir = None, prefix = "pre_app_")

['CHANNEL_TYPE', 'CODE_REJECT_REASON', 'FLAG_LAST_APPL_PER_CONTRACT', 'NAME_CASH_LOAN_PURPOSE', 'NAME_CLIENT_TYPE', 'NAME_CONTRACT_STATUS', 'NAME_CONTRACT_TYPE', 'NAME_GOODS_CATEGORY', 'NAME_PAYMENT_TYPE', 'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE', 'NAME_SELLER_INDUSTRY', 'NAME_TYPE_SUITE', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION', 'WEEKDAY_APPR_PROCESS_START']
['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE', 'CNT_PAYMENT', 'DAYS_DECISION', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_TERMINATION', 'HOUR_APPR_PROCESS_START', 'NFLAG_INSURED_ON_APPROVAL', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'SELLERPLACE_AREA']
